### A quantum algorithm to solve the Rubik's cube
Author's: Quentin Bragard, James Conway, Steve Flinter, Nicola Mariella, Conor McQuillan

Problem Statement: As part of the UCD Qiskit Community Summer Jam, we were challenged to solve a unique combinatorial problem using Quantum Computing Techniques
    
We propose a quantum algorithm for solving a 2x2x2 rubiks cube.

#### Description of the algorithm

<img src="Quantum Circuit (Rubix).png">

The **oracle** $O(A)$ encodes the state of the cube $A$ starting from $|0\rangle^{\otimes n}$.
Now, $\vec{A}$ is the vector containing the integer encoded colors of the stickers of the cube, so we define the oracle $O(\vec{A})$ as follows
$$ O(\vec{A}) = \left(\sum_{k=0}^{2^n-1} |k\rangle \langle k| e^{i 2\pi A_k/6}\right) H^{\otimes n}$$
where $A_k$ is the $k$th entry of $\vec{A}$. Also $n$ is the number of qubits necessary to represent the state, in our case $n=5$. Note that the oracle
is composed of a **Diagonal operator** (arXiv:0406176) and a **Walsh-Hadamard** transform. The inverse of the oracle is then
$$ O^{-1}(\vec{B}) = \left[O(\vec{B})\right]^{\dagger} = H^{\otimes n}\left(\sum_{k=0}^{2^n-1} |k\rangle \langle k| e^{-i 2\pi A_k/6}\right)$$

At this point we can observe that if the permutations between $O(\vec{A})$ and $O^{-1}(\vec{B})$ transform the state $\vec{A}$ to state $\vec{B}$, then the circuit
applied to the registry dedicated to the cube state is equivalent to the identity, i.e. the end state is $|0\rangle^{\otimes n}$. Moreover, since the representation of the
cube state uses $n$ qubits in superposition, a permutation implementing a solution determines a collapse to a single computational basis vector, hence when measured we obtain
a spike in the distribution. In regards to the measurement, if we filter the bit strings having the pattern $00000 p_1 p_2 \ldots p_t$, where the bits $p_i$ are the permutations
controlling bits, the spikes of such distribution are likely to be solutions, also the solution (i.e. the sequence of moves) is encoded in the bits $p_1 p_2 \ldots p_t$.

The permutation blocks are obtained by encoding the basic moves of the cube (eg. Up 90 deg, Right 90) into **permutation matrices**. The latters are built by re-arranging the columns
of the identity matrix of size $2^n \times 2^n$. Permutation matrices are unitary, indeed
$P^{T} = P^{-1}, P_{ij} \in \{0, 1\} \implies P^{T} = P^{H} \implies P^HP=PP^H=I$, hence $P$ is unitary. Such matrices are encoded directly using a **unitary operator** and by rely
on the **transpiler** for the translation to basic gates. The type and repetitions of permutations should be carefully choosen such that they generate the group of permutations allowed
by the Rubik's cube mechanism. 
Finally the unitary operators (corresponding to the permutations) are controlled by some qubits being initialized with the
Walsh-Hadamard transform, forming so the superposition of moves.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import widgets
from projection import Quaternion, project_points
from rubiks_cube import Cube, InteractiveCube
import random

Due to the limitation in number of qubits, we are not constructing a generator of the entire group of permutations of the cube.

For this reason the solver does not solve all starting cubes. As a proof of concept, we have tested [U], [R], [U,R], [R,U], [U,U] [U,U,U], [R,R], [R,R,R],[U,R,U],
U is the up move,
R is the right move,
[U,U] is the up move twice etc.

In [ ]:
%matplotlib qt  
N=2
c = Cube(N)
c._printCube()
c.draw_interactive()
plt.show()